# Set up

In [1]:
import os

os.environ['HADOOP_USER_NAME'] = 'hadoopuser'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.5 pyspark-shell'

In [2]:
import pyspark

In [10]:
from pyspark.sql.functions import from_json, col, unbase64

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

streaming_packages = [
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.5"
]

spark = (
    SparkSession.builder
    .appName("DebeziumCDCR Reader")
    .config("spark.sql.streaming.statefulOperator.checkCorrectness.enabled", "true") 
    .config("spark.jars.packages", ",".join(streaming_packages))
    .getOrCreate()
)

:: loading settings :: url = jar:file:/home/ad/Desktop/code/big-data-tools/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ad/.ivy2/cache
The jars for the packages stored in: /home/ad/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f36ff826-0967-4025-89e1-7a70fb10f459;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.5 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.5 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 151ms :: artifacts dl 6ms
	:: modules 

In [12]:
# decode Decimal value
s = 'CO96'

df = spark.createDataFrame(
    [('CO96',)], ['s_acctbal']
)

df.select(unbase64("s_acctbal")).show()

# df.show()

+-------------------+
|unbase64(s_acctbal)|
+-------------------+
|         [08 EF 7A]|
+-------------------+



In [ ]:
stream_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "tpch.public.supplier")
    .option("startingOffsets", "earliest")  # Or latest, earliest
    # .option("newRows", 1000)
    .load()
)

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, MapType, LongType, StructField

# Define the schema based on the provided JSON
schema = StructType([
    StructField("schema", StructType([
        StructField("type", StringType(), False),
        StructField("fields", StructType([
            StructField("type", StringType(), True),
            StructField("optional", BooleanType(), True),
            StructField("name", StringType(), True),
            StructField("version", IntegerType(), True),
            StructField("parameters", MapType(StringType(), StringType()), True),
            StructField("fields", ArrayType(StructType([
                StructField("type", StringType(), False),
                StructField("optional", BooleanType(), True),
                StructField("name", StringType(), True),
                StructField("version", IntegerType(), True),
                StructField("parameters", MapType(StringType(), StringType()), True),
                StructField("default", StringType(), True),
                StructField("fields", ArrayType(StructType([
                    StructField("type", StringType(), False),
                    StructField("optional", BooleanType(), True),
                    StructField("name", StringType(), True),
                    StructField("version", IntegerType(), True),
                    StructField("parameters", MapType(StringType(), StringType()), True),
                    StructField("default", StringType(), True),
                    StructField("field", StringType(), False)
                ])), True),
                StructField("field", StringType(), False)
            ])), True),
            StructField("field", StringType(), False)
        ]), True),
        StructField("optional", BooleanType(), False),
        StructField("name", StringType(), True),
        StructField("version", IntegerType(), True)
    ]), False),
    StructField("payload", StructType([
        StructField("before", StructType([
            StructField("s_suppkey", IntegerType(), False),
            StructField("s_name", StringType(), True),
            StructField("s_address", StringType(), True),
            StructField("s_nationkey", IntegerType(), True),
            StructField("s_phone", StringType(), True),
            StructField("s_acctbal", StringType(), True),
            StructField("s_comment", StringType(), True),
            StructField("s_dummy", StringType(), True)
        ]), True),
        StructField("after", StructType([
            StructField("s_suppkey", IntegerType(), False),
            StructField("s_name", StringType(), True),
            StructField("s_address", StringType(), True),
            StructField("s_nationkey", IntegerType(), True),
            StructField("s_phone", StringType(), True),
            StructField("s_acctbal", StringType(), True),
            StructField("s_comment", StringType(), True),
            StructField("s_dummy", StringType(), True)
        ]), True),
        StructField("source", StructType([
            StructField("version", StringType(), False),
            StructField("connector", StringType(), False),
            StructField("name", StringType(), False),
            StructField("ts_ms", LongType(), False),
            StructField("snapshot", StringType(), True),
            StructField("db", StringType(), False),
            StructField("sequence", StringType(), True),
            StructField("ts_us", LongType(), True),
            StructField("ts_ns", LongType(), True),
            StructField("schema", StringType(), False),
            StructField("table", StringType(), False),
            StructField("txId", LongType(), True),
            StructField("lsn", LongType(), True),
            StructField("xmin", LongType(), True)
        ]), False),
        StructField("transaction", StructType([
            StructField("id", StringType(), False),
            StructField("total_order", LongType(), False),
            StructField("data_collection_order", LongType(), False)
        ]), True),
        StructField("op", StringType(), False),
        StructField("ts_ms", LongType(), True),
        StructField("ts_us", LongType(), True),
        StructField("ts_ns", LongType(), True)
    ]), False)
])

# Extract the JSON value from the Kafka message and apply the schema
debezium_events_df = (
    stream_df
    .selectExpr("CAST(value AS STRING) AS json_value")
    .select(
        from_json(
            col("json_value"),
            schema=schema
        ).alias("data")
    )
    .select("data.payload.*")
)

# Now you can access the fields in the Debezium event
debezium_events_df.printSchema()

root
 |-- before: struct (nullable = true)
 |    |-- s_suppkey: integer (nullable = true)
 |    |-- s_name: string (nullable = true)
 |    |-- s_address: string (nullable = true)
 |    |-- s_nationkey: integer (nullable = true)
 |    |-- s_phone: string (nullable = true)
 |    |-- s_acctbal: string (nullable = true)
 |    |-- s_comment: string (nullable = true)
 |    |-- s_dummy: string (nullable = true)
 |-- after: struct (nullable = true)
 |    |-- s_suppkey: integer (nullable = true)
 |    |-- s_name: string (nullable = true)
 |    |-- s_address: string (nullable = true)
 |    |-- s_nationkey: integer (nullable = true)
 |    |-- s_phone: string (nullable = true)
 |    |-- s_acctbal: string (nullable = true)
 |    |-- s_comment: string (nullable = true)
 |    |-- s_dummy: string (nullable = true)
 |-- source: struct (nullable = true)
 |    |-- version: string (nullable = true)
 |    |-- connector: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- ts_ms: long

In [7]:
query = debezium_events_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

25/05/09 10:47:22 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4654f011-cd6a-4e53-8aab-3b84a3550f60. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/09 10:47:22 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/09 10:47:22 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/09 10:47:58 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)/ 1]
java.lang.OutOfMemoryError: Java heap space
	at java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:57)
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:335)
	at org.apache.spark.serializer.Seri

Py4JError: An error occurred while calling o59.getCause

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/ad/Desktop/code/big-data-tools/venv/lib/python3.12/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ad/Desktop/code/big-data-tools/venv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ad/Desktop/code/big-data-tools/venv/lib/python3.12/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
